In [ ]:
# tell python where the MUQ libraries are installed
import sys
sys.path.insert(0, "/my/muq/dir/build/install/lib/")

# import ploting tools
import matplotlib as mpl
#mpl.use('TKAgg')
import matplotlib.pyplot as plt

# import numpy, which we use for linear algebra in python
import numpy as np

import h5py

import random

# import the MUQ libraries
import pymuqUtilities as mu # import MUQ utilities module
import pymuqModeling as mm # import MUQ modeling module
import pymuqApproximationWrappers as ma # import MUQ approximation module
import pymuqSamplingAlgorithms as msa


In [ ]:
cov = np.array([[1.0,0.8],[0.8,1.5]]) # covariance

mu = np.array([.8, 2.3]) # mean
targetDensity = mm.Gaussian(mu, cov * 2.0).AsDensity()
problem = msa.SamplingProblem(targetDensity)


mu = np.array([1.0, 2.0]) # mean
targetDensity = mm.Gaussian(mu, cov).AsDensity()
problem2 = msa.SamplingProblem(targetDensity)

In [ ]:
nmcmc = 10000
trueCoeff = mu

# MCMC
options = dict()
options['NumSamples'] = nmcmc
options['PrintLevel'] = 3
options['KernelList'] = 'Kernel1'
options['Kernel1.Method'] = 'MHKernel'
options['Kernel1.Proposal'] = 'MyProposal'
options['Kernel1.MyProposal.Method'] = 'AMProposal'
options['Kernel1.MyProposal.InitialVariance'] = 0.1
options['Kernel1.MyProposal.AdaptSteps'] = 100
options['Kernel1.MyProposal.AdaptStart'] = 500

# create the MCMC sampler
mcmc = msa.SingleChainMCMC(options, problem2)
samps = mcmc.Run([trueCoeff])
print(samps.Mean())


# MLMCMC
mioptions = dict()
mioptions['NumSamples_0'] = nmcmc * 10
mioptions['NumSamples_1'] = nmcmc
mioptions['Subsampling'] = 5
mioptions['Proposal.Method'] = 'AMProposal'
mioptions['Proposal.InitialVariance'] = 0.1
mioptions['Proposal.AdaptSteps'] = 100
mioptions['Proposal.AdaptStart'] = 500

mimcmc = msa.MIMCMC(mioptions, trueCoeff, [problem2, problem])
mimcmc.Run([trueCoeff])
print(mimcmc.MeanParam())